# QData Quickstart Notebook

**Get started with QData in 5 minutes!**

This notebook demonstrates:
- Fetching cryptocurrency data (no API key needed)
- Fetching stock data with a free API key
- Basic data analysis and visualization
- Working with multiple providers

**Prerequisites**: `pip install qdata matplotlib`

## 1. Fetch Crypto Data (No API Key Required!)

CoinGecko is the easiest way to get started - no API key needed.

In [ ]:
from datetime import datetime, timedelta

from qdata.providers import CoinGeckoProvider

# Calculate date range (last 30 days)
end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")

print(f"Fetching Bitcoin data from {start_date} to {end_date}...")

# Create provider (no API key needed!)
provider = CoinGeckoProvider()

# Fetch Bitcoin data
btc_data = provider.fetch_ohlcv("bitcoin", start_date, end_date)

print(f"\n✅ Successfully fetched {len(btc_data)} days of Bitcoin data!")
btc_data.head()

## 2. Visualize Price Data

Let's create a simple price chart with matplotlib.

In [ ]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt

# Convert Polars DataFrame to pandas for plotting
btc_pd = btc_data.to_pandas()

# Create figure
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Plot 1: Price with High/Low bands
ax1.plot(btc_pd["timestamp"], btc_pd["close"], label="Close", color="blue", linewidth=2)
ax1.fill_between(
    btc_pd["timestamp"], btc_pd["low"], btc_pd["high"], alpha=0.2, label="Daily Range", color="gray"
)
ax1.set_ylabel("Price (USD)", fontsize=12)
ax1.set_title("Bitcoin Price (Last 30 Days)", fontsize=14, fontweight="bold")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Volume
ax2.bar(btc_pd["timestamp"], btc_pd["volume"], color="steelblue", alpha=0.6)
ax2.set_ylabel("Volume (USD)", fontsize=12)
ax2.set_xlabel("Date", fontsize=12)
ax2.set_title("Trading Volume", fontsize=14, fontweight="bold")
ax2.grid(True, alpha=0.3)

# Format x-axis
ax2.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("\n📊 Price Stats:")
print(f"  High:   ${btc_pd['high'].max():,.2f}")
print(f"  Low:    ${btc_pd['low'].min():,.2f}")
print(f"  Latest: ${btc_pd['close'].iloc[-1]:,.2f}")

## 3. Calculate Returns and Volatility

Let's calculate some basic metrics.

In [ ]:
# Calculate daily returns
btc_data = btc_data.with_columns(
    ((btc_data["close"] - btc_data["close"].shift(1)) / btc_data["close"].shift(1) * 100).alias(
        "daily_return_pct"
    )
)

# Calculate rolling volatility (7-day)
btc_data = btc_data.with_columns(
    btc_data["daily_return_pct"].rolling_std(window_size=7).alias("volatility_7d")
)

print("📈 Returns Analysis (Last 30 Days):")
print(f"  Total Return:     {btc_data['daily_return_pct'].sum():.2f}%")
print(f"  Best Day:         {btc_data['daily_return_pct'].max():.2f}%")
print(f"  Worst Day:        {btc_data['daily_return_pct'].min():.2f}%")
print(f"  Avg Daily Return: {btc_data['daily_return_pct'].mean():.2f}%")
print(f"  Volatility (std): {btc_data['daily_return_pct'].std():.2f}%")

# Show data with new columns
btc_data.select(["timestamp", "close", "daily_return_pct", "volatility_7d"]).tail(10)

## 4. Fetch Stock Data (Requires Free API Key)

For stocks, you'll need a free API key from [Tiingo](https://api.tiingo.com/account/api/token).

**Get free key**: https://api.tiingo.com/account/api/token (1000 calls/day free)

In [ ]:
import os

# Check for API key
tiingo_key = os.getenv("TIINGO_API_KEY")

if not tiingo_key:
    print("⚠️  TIINGO_API_KEY not set!")
    print("   Get free key at: https://api.tiingo.com/account/api/token")
    print("   Then set: export TIINGO_API_KEY='your_key_here'")
else:
    from qdata.providers import TiingoProvider

    # Create provider
    tiingo = TiingoProvider(api_key=tiingo_key)

    # Fetch Apple stock data
    aapl_data = tiingo.fetch_ohlcv("AAPL", start_date, end_date)

    print(f"✅ Fetched {len(aapl_data)} days of AAPL data")
    aapl_data.head()

## 5. Compare Multiple Assets

Let's compare Bitcoin and Apple stock performance.

In [ ]:
if tiingo_key:
    # Normalize prices to 100 at start
    btc_pd = btc_data.to_pandas()
    aapl_pd = aapl_data.to_pandas()

    btc_pd["normalized"] = btc_pd["close"] / btc_pd["close"].iloc[0] * 100
    aapl_pd["normalized"] = aapl_pd["close"] / aapl_pd["close"].iloc[0] * 100

    # Plot comparison
    plt.figure(figsize=(14, 6))
    plt.plot(btc_pd["timestamp"], btc_pd["normalized"], label="Bitcoin", linewidth=2)
    plt.plot(aapl_pd["timestamp"], aapl_pd["normalized"], label="Apple (AAPL)", linewidth=2)
    plt.axhline(y=100, color="gray", linestyle="--", alpha=0.5)
    plt.ylabel("Normalized Price (Start = 100)", fontsize=12)
    plt.xlabel("Date", fontsize=12)
    plt.title("Bitcoin vs Apple Stock (Normalized)", fontsize=14, fontweight="bold")
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    print("\n📊 Performance Comparison:")
    btc_return = (btc_pd["close"].iloc[-1] / btc_pd["close"].iloc[0] - 1) * 100
    aapl_return = (aapl_pd["close"].iloc[-1] / aapl_pd["close"].iloc[0] - 1) * 100
    print(f"  Bitcoin: {btc_return:+.2f}%")
    print(f"  AAPL:    {aapl_return:+.2f}%")
else:
    print("⚠️  Set TIINGO_API_KEY to run comparison")

## 6. Try Multiple Cryptocurrencies

CoinGecko supports 10,000+ cryptocurrencies!

In [ ]:
# Fetch multiple cryptocurrencies
cryptos = ["bitcoin", "ethereum", "cardano", "ripple"]
crypto_data = {}

for crypto in cryptos:
    data = provider.fetch_ohlcv(crypto, start_date, end_date)
    crypto_data[crypto] = data
    print(f"✅ {crypto.capitalize()}: {len(data)} days")

# Create comparison chart
plt.figure(figsize=(14, 6))

for crypto, data in crypto_data.items():
    df = data.to_pandas()
    normalized = df["close"] / df["close"].iloc[0] * 100
    plt.plot(df["timestamp"], normalized, label=crypto.capitalize(), linewidth=2)

plt.axhline(y=100, color="gray", linestyle="--", alpha=0.5)
plt.ylabel("Normalized Price (Start = 100)", fontsize=12)
plt.xlabel("Date", fontsize=12)
plt.title("Cryptocurrency Comparison (Normalized)", fontsize=14, fontweight="bold")
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 30-Day Returns:")
for crypto, data in crypto_data.items():
    df = data.to_pandas()
    ret = (df["close"].iloc[-1] / df["close"].iloc[0] - 1) * 100
    print(f"  {crypto.capitalize():12s}: {ret:+7.2f}%")

## Next Steps

**Learn More**:
- [Provider Selection Guide](../../docs/provider-selection-guide.md) - Choose the right data source
- [Tutorial 02: Rate Limiting](../../docs/tutorials/02_rate_limiting.md) - Avoid API bans
- [Tutorial 03: Incremental Updates](../../docs/tutorials/03_incremental_updates.md) - Save API calls

**Try More Notebooks**:
- `02_provider_comparison.ipynb` - Compare data quality across providers
- `03_incremental_updates.ipynb` - Smart data pipeline patterns
- `04_multi_asset_portfolio.ipynb` - Build a diversified portfolio
- `05_data_quality.ipynb` - Validate and clean market data

**Happy trading!** 🚀